In [2]:
from libpgm.nodedata import NodeData
from libpgm.graphskeleton import GraphSkeleton
from libpgm.discretebayesiannetwork import DiscreteBayesianNetwork
from libpgm.lgbayesiannetwork import LGBayesianNetwork
from libpgm.hybayesiannetwork import HyBayesianNetwork
from libpgm.dyndiscbayesiannetwork import DynDiscBayesianNetwork
from libpgm.tablecpdfactorization import TableCPDFactorization
from libpgm.sampleaggregator import SampleAggregator
from libpgm.pgmlearner import PGMLearner

import numpy as np
#create table
from prettytable import PrettyTable

#read csv
import pandas as pd

In [3]:
#data set dopo il processo di undersampling con weka
data=pd.read_csv("./csv/training_sentimental_undersampling.csv")
del data['id']
data['id'] = np.arange(len(data))
cols=data.columns.tolist()
cols= cols[-1:]+cols[:-1]
data=data[cols]
all_data=len(data)
data.head()

,id,hotel,reader,helpful,overall,value,room,location,clean,check,service,business,target
0,0,hotel_305810,2,1,3,3,3,3,3,3,3,3,good
1,1,hotel_572859,10,10,5,5,5,5,5,5,5,5,good
2,2,hotel_230875,-1,-1,3,3,3,5,5,3,3,4,good
3,3,hotel_234752,1,0,5,5,3,5,5,4,4,3,good
4,4,hotel_228621,-1,-1,5,5,5,4,5,3,4,3,good


In [4]:
all_data=len(data)
bad=data.query('target == "bad" and overall >0')
good=data.query('target == "good"')
bad_n=len(bad)
good_n=len(good)

print(float(good_n)/all_data,float(bad_n)/all_data)

(0.5651997291807719, 0.4348002708192282)


In [5]:
#numero di volte per ogni valore di value dato good
value_g_1=data.query('target == "good" and value == 1')
value_g_2=data.query('target == "good" and value == 2')
value_g_3=data.query('target == "good" and value == 3')
value_g_4=data.query('target == "good" and value == 4')
value_g_5=data.query('target == "good" and value == 5')
value_g=[len(value_g_1),len(value_g_2),len(value_g_3),len(value_g_4),len(value_g_5)]

#numero di volte per ogni valore di value dato bad
value_b_1=data.query('target == "bad" and value == 1')
value_b_2=data.query('target == "bad" and value == 2')
value_b_3=data.query('target == "bad" and value == 3')
value_b_4=data.query('target == "bad" and value == 4')
value_b_5=data.query('target == "bad" and value == 5')
value_b=[len(value_b_1),len(value_b_2),len(value_b_3),len(value_b_4),len(value_b_5)]

#numero di volte per ogni valore di location dato value =1
loc_v1_1=data.query('value == 1 and location == 1')
loc_v1_2=data.query('value == 1 and location == 2')
loc_v1_3=data.query('value == 1 and location == 3')
loc_v1_4=data.query('value == 1 and location == 4')
loc_v1_5=data.query('value == 1 and location == 5')
loc_v1=[len(loc_v1_1),len(loc_v1_2),len(loc_v1_3),len(loc_v1_4),len(loc_v1_5)]
loc_v1

#numero di volte per ogni valore di location dato value =2
loc_v2_1=data.query('value == 2 and location == 1')
loc_v2_2=data.query('value == 2 and location == 2')
loc_v2_3=data.query('value == 2 and location == 3')
loc_v2_4=data.query('value == 2 and location == 4')
loc_v2_5=data.query('value == 2 and location == 5')
loc_v2=[len(loc_v2_1),len(loc_v2_2),len(loc_v2_3),len(loc_v2_4),len(loc_v2_5)]
loc_v2

#numero di volte per ogni valore di location dato value =3
loc_v3_1=data.query('value == 3 and location == 1')
loc_v3_2=data.query('value == 3 and location == 2')
loc_v3_3=data.query('value == 3 and location == 3')
loc_v3_4=data.query('value == 3 and location == 4')
loc_v3_5=data.query('value == 3 and location == 5')
loc_v3=[len(loc_v3_1),len(loc_v3_2),len(loc_v3_3),len(loc_v3_4),len(loc_v3_5)]
loc_v3

#numero di volte per ogni valore di location dato value =4
loc_v4_1=data.query('value == 4 and location == 1')
loc_v4_2=data.query('value == 4 and location == 2')
loc_v4_3=data.query('value == 4 and location == 3')
loc_v4_4=data.query('value == 4 and location == 4')
loc_v4_5=data.query('value == 4 and location == 5')
loc_v4=[len(loc_v4_1),len(loc_v4_2),len(loc_v4_3),len(loc_v4_4),len(loc_v4_5)]
loc_v4

def calculate_cpt(child,parent,val_p):
    q1=data.query(parent+' == '+val_p+' and '+child+' == 1')
    q2=data.query(parent+' == '+val_p+' and '+child+' == 2')
    q3=data.query(parent+' == '+val_p+' and '+child+' == 3')
    q4=data.query(parent+' == '+val_p+' and '+child+' == 4')
    q5=data.query(parent+' == '+val_p+' and '+child+' == 5')
    r=[len(q1),len(q2),len(q3),len(q4),len(q5)]
    return r

In [6]:
value_location=[]
for i in range(1,6):
    value_location.append(calculate_cpt('location','value',str(i)))
value_location
value_service=[]
for i in range(1,6):
    value_service.append(calculate_cpt('service','value',str(i)))
value_service

value_room=[]
for i in range(1,6):
    value_room.append(calculate_cpt('room','value',str(i)))
value_room

service_clean=[]
for i in range(1,6):
    service_clean.append(calculate_cpt('clean','service',str(i)))
service_clean

service_check=[]
for i in range(1,6):
    service_check.append(calculate_cpt('check','service',str(i)))
service_check

#create view to cpt value
def generate_cpt_bin(name,array_g,array_b):
    cpt= PrettyTable()
    cpt.field_names =["P("+name+"|target)",1,2,3,4,5]
    good_c=['good']
    bad_c=['bad']
    for i in range(len(array_g)):
        good_c.append(float(array_g[i])/good_n)
        bad_c.append(float(array_b[i])/bad_n)
    cpt.add_row(good_c)
    cpt.add_row(bad_c)
    return cpt
print(generate_cpt_bin("value",value_g,value_b))

+-----------------+-----------------+-----------------+----------------+----------------+-----------------+
| P(value|target) |        1        |        2        |       3        |       4        |        5        |
+-----------------+-----------------+-----------------+----------------+----------------+-----------------+
|       good      | 0.0258744609487 | 0.0570196454241 | 0.119789171059 | 0.319357930043 |  0.477958792525 |
|       bad       |  0.460292743694 |  0.251635004671 | 0.141388975397 | 0.078168794768 | 0.0685144814699 |
+-----------------+-----------------+-----------------+----------------+----------------+-----------------+


In [7]:
def num_node(name_node):
    support=[]
    for i in range(1,6):
        support.append(len(data.query(name_node+' == '+str(i))))
    return support

#create view to cpt 5x5
def generate_cpt(child,parent,array_query):
    num_p=num_node(parent)
    cpt= PrettyTable()
    cpt.field_names =["P("+child+"|"+parent+")",1,2,3,4,5]
    one=['one']
    two=['two']
    three=['three']
    four=['four']
    five=['five']
    for i in range(len(array_query)):
        one.append(float(array_query[0][i])/num_p[0])
        two.append(float(array_query[1][i])/num_p[1])
        three.append(float(array_query[2][i])/num_p[2])
        four.append(float(array_query[3][i])/num_p[3])
        five.append(float(array_query[4][i])/num_p[4])
    cpt.add_row(one)
    cpt.add_row(two)
    cpt.add_row(three)
    cpt.add_row(four)
    cpt.add_row(five)
    return cpt

In [11]:
cpt_v_l=generate_cpt('location','value',value_location)
print(cpt_v_l)

cpt_v_r=generate_cpt('room','value',value_room)
print(cpt_v_r)

cpt_v_s=generate_cpt('service','value',value_service)
print(cpt_v_s)

cpt_s_c=generate_cpt('clean','service',service_clean)
print(cpt_s_c)

cpt_s_check=generate_cpt('check','service',service_check)
print(cpt_s_check)

+-------------------+------------------+------------------+-----------------+----------------+----------------+
| P(location|value) |        1         |        2         |        3        |       4        |       5        |
+-------------------+------------------+------------------+-----------------+----------------+----------------+
|        one        |  0.21815889029   |  0.211223203026  |  0.271752837327 | 0.192938209332 | 0.105926860025 |
|        two        | 0.0774378585086  |  0.166347992352  |  0.285850860421 | 0.318355640535 | 0.152007648184 |
|       three       | 0.0314465408805  |  0.107966457023  |  0.214884696017 | 0.375262054507 | 0.270440251572 |
|        four       | 0.00505050505051 | 0.0296717171717  |  0.107954545455 | 0.335227272727 | 0.522095959596 |
|        five       | 0.00135440180587 | 0.00722347629797 | 0.0379232505643 | 0.193227990971 | 0.760270880361 |
+-------------------+------------------+------------------+-----------------+----------------+----------

In [12]:
data_test=pd.read_csv("./csv/testing_sentimental.csv")
data_test.head()

,id,hotel,reader,helpful,overall,value,room,location,clean,check,service,business,target
0,0,hotel_80799,-1,-1,3,5,3,3,3,5,3,4,good
1,1,hotel_80799,1,1,1,1,1,1,1,3,2,1,good
2,2,hotel_80799,2,1,4,5,3,4,5,5,5,3,good
3,3,hotel_80799,-1,-1,1,1,1,1,3,2,3,2,bad
4,4,hotel_80799,3,3,1,2,1,3,2,5,5,1,bad


In [13]:
skel= GraphSkeleton()
nd =NodeData()
skel.load('./skeleton_bn_sentimental.txt')
nd.load("./skeleton_bn_sentimental.txt")

skel.toporder()
bn=DiscreteBayesianNetwork(skel, nd)


def inference(row):
    fn =TableCPDFactorization(bn)
    evidence =dict(clean=row[8],value=row[5],room=row[6],location=row[7],check=row[9],service=row[10])
    query= {'target' :['bad']}
    bad = fn.specificquery(query,evidence)
    good=1-bad
    return (good,bad)

list_good=[]
list_result=[]
for i in range(len(data_test)):
    row=data_test.query('id=='+str(i))
    result_row=row.values.tolist()[0]
    good,bad=inference(result_row)
    list_good.append(good)
    print(good,bad)
    if good < bad:
        list_result.append('bad')
    else:
        list_result.append('good')
data_test['target']=list_result

(0.9006772009029669, 0.09932279909703316)
(0.06809583858750623, 0.9319041614124938)
(0.9006772009029669, 0.09932279909703316)
(0.06809583858750623, 0.9319041614124938)
(0.22753346080351067, 0.7724665391964893)
(0.06809583858750623, 0.9319041614124938)
(0.06809583858750623, 0.9319041614124938)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.22753346080351067, 0.7724665391964893)
(0.8415404040403267, 0.15845959595967327)
(0.22753346080351067, 0.7724665391964893)
(0.06809583858750623, 0.9319041614124938)
(0.06809583858750623, 0.9319041614124938)
(0.06809583858750623, 0.9319041614124938)
(0.5241090146753131, 0.4758909853246868)
(0.06809583858750623, 0.9319041614124938)
(0.22753346080351067, 0.7724665391964893)
(0.22753346080351067, 0.7724665391964893)
(0.06809583858750623, 0.9319041614124938)
(0.9006772009029669, 0.099322799097

(0.8415404040403267, 0.15845959595967327)
(0.22753346080351067, 0.7724665391964893)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.5241090146753131, 0.4758909853246868)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.22753346080351067, 0.7724665391964893)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.22753346080351067, 0.7724665391964893)
(0.9006772009029669, 0.09932279909703316)
(0.22753346080351067, 0.7724665391964893)
(0.9006772009029669, 0.09932279909703316)
(0.22753346080351067, 0.7724665391964893)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.158459595959

(0.5241090146753131, 0.4758909853246868)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.22753346080351067, 0.7724665391964893)
(0.22753346080351067, 0.7724665391964893)
(0.5241090146753131, 0.4758909853246868)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.06809583858750623, 0.9319041614124938)
(0.9006772009029669, 0.09932279909703316)
(0.5241090146753131, 0.4758909853246868)
(0.5241090146753131, 0.4758909853246868)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.06809583858750623, 0.9319041614124938)
(0.5241090146753131, 0.475890985324686

(0.9006772009029669, 0.09932279909703316)
(0.06809583858750623, 0.9319041614124938)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.06809583858750623, 0.9319041614124938)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.5241090146753131, 0.4758909853246868)
(0.9006772009029669, 0.09932279909703316)
(0.22753346080351067, 0.7724665391964893)
(0.5241090146753131, 0.4758909853246868)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.22753346080351067, 0.7724665391964893)
(0.9006772009029669, 0.0993227990970

(0.22753346080351067, 0.7724665391964893)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.5241090146753131, 0.4758909853246868)
(0.5241090146753131, 0.4758909853246868)
(0.22753346080351067, 0.7724665391964893)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.5241090146753131, 0.4758909853246868)
(0.8415404040403267, 0.15845959595967327)
(0.5241090146753131, 0.4758909853246868)
(0.5241090146753131, 0.4758909853246868)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.5241090146753131, 0.4758909853246868)
(0.5241090146753131, 0.4758909853246868)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)

...

(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.22753346080351067, 0.7724665391964893)
(0.8415404040403267, 0.15845959595967327)
(0.22753346080351067, 0.7724665391964893)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.5241090146753131, 0.4758909853246868)
(0.22753346080351067, 0.7724665391964893)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.22753346080351067, 0.7724665391964893)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.5241090146753131, 0.4758909853246868)
(0.9006772009029669, 0.09932279909703316)
(0.22753346080351067, 0.7724665391964893)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.0993227990970

(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.22753346080351067, 0.7724665391964893)
(0.22753346080351067, 0.7724665391964893)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.5241090146753131, 0.4758909853246868)
(0.5241090146753131, 0.4758909853246868)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.5241090146753131, 0.4758909853246868)
(0.8415404040403267, 0.15845959595967327)
(0.5241090146753131, 0.4758909853246868)
(0.22753346080351067, 0.7724665391964893)
(0.9006772009029669, 0.099322799097033

(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.22753346080351067, 0.7724665391964893)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.22753346080351067, 0.7724665391964893)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.5241090146753131, 0.4758909853246868)
(0.9006772009029669, 0.09932279909703316)
(0.5241090146753131, 0.4758909853246868)
(0.9006772009029669, 0.09932279909703316)
(0.06809583858750623, 0.9319041614124938)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.5241090146753131, 0.4758909853246868)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.5241090146753131, 0.4758909853246868)
(0.8415404040403267, 0.158459595959673

(0.5241090146753131, 0.4758909853246868)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.22753346080351067, 0.7724665391964893)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.5241090146753131, 0.4758909853246868)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.22753346080351067, 0.7724665391964893)
(0.5241090146753131, 0.4758909853246868)
(0.5241090146753131, 0.4758909853246868)
(0.5241090146753131, 0.4758909853246868)
(0.8415404040403267, 0.15845959595967327)
(0.5241090146753131, 0.4758909853246868)
(0.5241090146753131, 0.4758909853246868)

(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.5241090146753131, 0.4758909853246868)
(0.8415404040403267, 0.15845959595967327)
(0.22753346080351067, 0.7724665391964893)
(0.22753346080351067, 0.7724665391964893)
(0.9006772009029669, 0.09932279909703316)
(0.5241090146753131, 0.4758909853246868)
(0.8415404040403267, 0.15845959595967327)
(0.5241090146753131, 0.4758909853246868)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.5241090146753131, 0.4758909853246868)
(0.8415404040403267, 0.15845959595967327)
(0.5241090146753131, 0.4758909853246868)
(0.8415404040403267, 0.15845959595967327)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.8415404040403267, 0.15845959595967327)
(0.9006772009029669, 0.09932279909703316)
(0.9006772009029669, 0.09932279909703316)
(0.5241090146753131, 0.4758909853246868

In [14]:
data_test['prob']=list_good
data_test.to_csv("./csv/result_sentimental/testing_sentimental_netbayesain.csv",index=False)